In [60]:
import pandas as pd
import re
import string
import numpy as np

In [3]:
df = pd.read_csv("labeled_data.csv")

In [ ]:
df

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [6]:
df = df.drop(['Unnamed: 0','count', 'hate_speech', 'offensive_language', 'neither'], axis=1)

In [7]:
df['tweet']= df['tweet'].apply(lambda x: x.lower())

In [8]:
df

,class,tweet
0,2,!!! rt @mayasolovely: as a woman you shouldn't...
1,1,!!!!! rt @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...
3,1,!!!!!!!!! rt @c_g_anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! rt @shenikaroberts: the shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@lifeasking: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like i ain...
24781,1,youu got wild bitches tellin you lies


In [9]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [10]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [11]:
def preprocessing(text):
    output = " ".join([i for i in text.split() if i not in stopwords])
    punctuationfree = "".join([i for i in output if i not in string.punctuation])
    lemm_text = " ".join([wordnet_lemmatizer.lemmatize(word) for word in punctuationfree.split()])
    return lemm_text

df['tweet']= df['tweet'].apply(lambda x:preprocessing(x))

In [ ]:
df

In [142]:
# !pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 26.6 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.0 MB/s  eta 0:00:01


### Word2Vec

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

In [13]:
text = '. '.join(df['tweet'])

In [ ]:
text

In [15]:
data = []
for i in sent_tokenize(text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [ ]:
data

In [17]:
model1 = gensim.models.Word2Vec(data, min_count=1,
                                vector_size=100, window=5)

In [33]:
print(model1)

Word2Vec<vocab=39151, vector_size=100, alpha=0.025>


In [18]:
model2 = gensim.models.FastText(data, vector_size=100, window=5, min_count=1)

In [19]:
model2.wv.get_vector('bad')

array([-1.0429167e+00,  1.8856901e+00, -1.2663317e-01, -2.2378220e-01,
        5.2460223e-01, -4.7055796e-01,  9.1117585e-01,  1.0626067e+00,
        1.3137904e+00, -1.3334157e+00, -4.9269298e-01,  3.3759028e-01,
       -1.5613754e+00,  2.4414299e+00, -8.6124569e-01,  5.6652683e-01,
        5.3977221e-01,  2.1900105e-01,  9.3020517e-03, -1.8471794e+00,
       -1.3062351e+00,  9.6717548e-01, -3.4825140e-01,  6.4869243e-01,
       -1.2568403e+00, -3.3689046e-01, -7.4688351e-01,  1.2027250e-01,
        2.1966293e+00, -1.2843390e+00, -7.3803532e-01, -1.4432801e-01,
        1.1612551e+00, -4.0326198e-03,  2.1145828e-01,  1.2964697e-01,
        5.6547129e-01,  8.1161278e-01, -1.9541128e+00,  5.6700606e-02,
        7.5918353e-01, -1.3986348e-01,  5.3739446e-01, -1.7368630e+00,
        3.9722580e-01,  5.2114107e-02, -1.2709125e+00, -8.8171762e-01,
       -3.7032459e-02,  3.7594754e-01,  7.2564739e-01, -6.3473684e-01,
        1.0126089e+00, -6.2753427e-01, -1.7840202e-01, -2.0798226e-01,
      

In [ ]:
df['tweet']

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence

### CNN

In [27]:
words = ["<PAD>"]
x = []
for sent in df['tweet']:
    tmp = []
    for w in sent.split(' '):
        if w not in words:
            words.append(w)
        tmp.append(words.index(w))
    x.append(torch.tensor(tmp))

In [28]:
x = pad_sequence(x, batch_first=True, padding_value=0)
x.shape

torch.Size([24783, 30])

In [52]:
class CNN(nn.Module):
    def __init__(self, vocab_dim):
        super(CNN, self).__init__()
        emb_dim = 100
        self.emb = nn.Embedding(vocab_dim, emb_dim)
        self.conv = nn.Conv1d(emb_dim, 64, 3)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(64, 2)
    
    def forward(self, x):
        x = self.emb(x)
        x = self.conv(x.permute(0,2,1))
        x = self.pool(torch.relu(x)).squeeze(2)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [53]:
mod = CNN(len(words))
out = mod(x)

In [54]:
out.shape

torch.Size([24783, 2])

In [55]:
y = df['class'].copy()
y[y==2] = 1
y = torch.from_numpy(pd.get_dummies(y).values)
y = y.float()

In [56]:
y

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [57]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=0.01)
for epoch in range(5):
    mod.train()
    optimizer.zero_grad()
    
    out = mod(x)
    loss = criterion(out, y)
    
    loss.backward()
    optimizer.step()
    
    print("epoch:",epoch, " loss:", loss, "prediction:", out[:5,:])


epoch: 0  loss: tensor(0.6703, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.4083, 0.3638],
        [0.4600, 0.4696],
        [0.4371, 0.5107],
        [0.3513, 0.5809],
        [0.4477, 0.4821]], grad_fn=<SliceBackward0>)
epoch: 1  loss: tensor(0.2396, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.1505, 0.8561],
        [0.2299, 0.8778],
        [0.1116, 0.9163],
        [0.1295, 0.9495],
        [0.1269, 0.9142]], grad_fn=<SliceBackward0>)
epoch: 2  loss: tensor(0.2617, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.0164, 0.9889],
        [0.0242, 0.9901],
        [0.0099, 0.9935],
        [0.0132, 0.9954],
        [0.0099, 0.9936]], grad_fn=<SliceBackward0>)
epoch: 3  loss: tensor(0.3042, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.0067, 0.9961],
        [0.0089, 0.9965],
        [0.0036, 0.9981],
        [0.0061, 0.9985],
        [0.0036, 0.9981]], grad_fn=<SliceBackward0>)
epoch: 4  loss: tensor(0.2946, grad_fn=<Bina

In [64]:
y_true = np.argmax(y, axis=1)
y_pred = np.argmax(out.detach().numpy(), axis=1)

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_true)

0.9422991566799822

### RNN

In [78]:
class RNN(nn.Module):
    def __init__(self, vocab_dim, seq_len):
        super(RNN, self).__init__()
        emb_dim = 100
        self.emb = nn.Embedding(vocab_dim, emb_dim)
        self.rnn = nn.RNN(emb_dim, 100)
        self.fc = nn.Linear(100, 2)
    
    def forward(self, x):
        x = self.emb(x)
        _,x = self.rnn(x.permute(1,0,2))
        x = self.fc(x).squeeze(0)
        return torch.sigmoid(x)

In [79]:
mod = RNN(len(words), x.shape[1])
out = mod(x)
out.shape

torch.Size([24783, 2])

In [80]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=0.01)

In [81]:
for epoch in range(5):
    mod.train()
    optimizer.zero_grad()
    
    out = mod(x)
    loss = criterion(out, y)
    
    loss.backward()
    optimizer.step()
    
    print("epoch:",epoch, " loss:", loss, "prediction:", out[:5,:])

epoch: 0  loss: tensor(0.6396, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.4988, 0.5636],
        [0.4988, 0.5636],
        [0.4988, 0.5636],
        [0.4988, 0.5636],
        [0.4988, 0.5636]], grad_fn=<SliceBackward0>)
epoch: 1  loss: tensor(0.2413, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.0714, 0.8530],
        [0.0714, 0.8530],
        [0.0714, 0.8530],
        [0.0714, 0.8530],
        [0.0714, 0.8530]], grad_fn=<SliceBackward0>)
epoch: 2  loss: tensor(0.2212, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.0580, 0.9532],
        [0.0580, 0.9532],
        [0.0580, 0.9532],
        [0.0580, 0.9532],
        [0.0580, 0.9532]], grad_fn=<SliceBackward0>)
epoch: 3  loss: tensor(0.2261, grad_fn=<BinaryCrossEntropyBackward0>) prediction: tensor([[0.0484, 0.9696],
        [0.0484, 0.9696],
        [0.0484, 0.9696],
        [0.0484, 0.9696],
        [0.0484, 0.9696]], grad_fn=<SliceBackward0>)
epoch: 4  loss: tensor(0.2279, grad_fn=<Bina

In [76]:
y_true = np.argmax(y, axis=1)
y_pred = np.argmax(out.detach().numpy(), axis=1)